This notebook is for preparing input for mixQTL call.
It is adapted from mixQTL_inputs.ipynb by Francois.

In [1]:
import numpy as np
import gzip
import pandas as pd
# import pyarrow

In [2]:
datadir = '/gpfs/data/im-lab/nas40t2/yanyul/GEUVADIS-mixqtl'

# input
vcf = f'{datadir}/tempo-GEUVADIS.chr22.PH1PH2_465.IMPFRQFILT_BIALLELIC_PH.annotv2.genotypes.vcf.gz'


# output
hap1_file = f'{datadir}/tempo-GEUVADIS.chr22.PH1PH2_465.IMPFRQFILT_BIALLELIC_PH.annotv2.genotypes.hap1.parquet'
hap2_file = f'{datadir}/tempo-GEUVADIS.chr22.PH1PH2_465.IMPFRQFILT_BIALLELIC_PH.annotv2.genotypes.hap2.parquet'

In [3]:
# assuming 0|0, 0|1, 1|0, 1|1, .|., code per-haplotype dosages as 0, 1, or -1 (missing)

gt_dict = {'0':np.int8(0), '1':np.int8(1), '.':np.int8(-1)}

with gzip.open(vcf, 'rt') as f:
    variant_ids = []
    hap1 = []
    hap2 = []
    for line in f:
        if line.startswith('##'):
            continue
        break
    sample_ids = line.strip().split('\t')[9:]

    # read first line, parse field
    line = f.readline().strip().split('\t')
    gt_ix = line[8].split(':').index('GT')
    variant_ids.append(line[2])
    d = [i.split(':')[gt_ix].split('|') if len(i.split(':')[gt_ix].split('|')) == 2 else ['.', '.'] for i in line[9:]]
    hap1.append(np.array([gt_dict[i[0]] for i in d]))
    hap2.append(np.array([gt_dict[i[1]] for i in d]))

    for k,line in enumerate(f,2):
        line = line.strip().split('\t')
        variant_ids.append(line[2])
        d = [i.split(':')[gt_ix].split('|') if len(i.split(':')[gt_ix].split('|')) == 2 else ['.', '.'] for i in line[9:]]
        hap1.append(np.array([gt_dict[i[0]] for i in d]))
        hap2.append(np.array([gt_dict[i[1]] for i in d]))

        if np.mod(k,1000)==0:
            print('\rVariants parsed: {}'.format(k), end='')


FileNotFoundError: [Errno 2] No such file or directory: '/gpfs/data/im-lab/nas40t2/yanyul/GEUVADIS-mixqtl/tempo-GEUVADIS.chr22.PH1PH2_465.IMPFRQFILT_BIALLELIC_PH.annotv2.genotypes.vcf.gz'

In [ ]:
hap1_df = pd.DataFrame(np.array(hap1), index=variant_ids, columns=sample_ids)
hap1_df.to_parquet(hap1_file)  # , engine='fastparquet')

hap2_df = pd.DataFrame(np.array(hap2), index=variant_ids, columns=sample_ids)
hap2_df.to_parquet(hap2_file)  # , engine='fastparquet')